In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_524583/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 16:39:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 16:39:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Orders table
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'customer_id': [1, 1, 1, 1, 1, 2, 2, 3, 3],
    'order_date': ['2019-07-01', '2019-11-01', '2020-05-26', '2021-08-31', '2022-12-07',
                   '2015-01-01', '2017-11-07', '2017-01-01', '2018-11-07'],
    'price': [1100, 1200, 3000, 3100, 4700, 700, 1000, 900, 900]
}

# Create DataFrame for Orders
orders_df = pd.DataFrame(orders_data)

# Convert order_date to datetime type
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

# Display the DataFrame
print("Orders table:")
print(orders_df)



df_person = spark.createDataFrame(orders_df)
df_person.createOrReplaceTempView("Orders")



Orders table:
   order_id  customer_id order_date  price
0         1            1 2019-07-01   1100
1         2            1 2019-11-01   1200
2         3            1 2020-05-26   3000
3         4            1 2021-08-31   3100
4         5            1 2022-12-07   4700
5         6            2 2015-01-01    700
6         7            2 2017-11-07   1000
7         8            3 2017-01-01    900
8         9            3 2018-11-07    900


In [22]:
query = """
    with cus_year as (
        select
            customer_id,
            explode(sequence(min_year, max_year)) as year
        from (
            select
                distinct
                customer_id,
                min_year,
                max_year
            from (
                select
                *,
                    min(year(order_date)) over (partition by customer_id) as min_year,
                    max(year(order_date)) over (partition by customer_id) as max_year
                from Orders o 
            )
        )
    ),
    
    cus_year_price as (
        select
            customer_id,
            year,
            sum(price) as price
        from (
            select
                customer_id,
                year(order_date) as year,
                price
            from Orders o 
        ) group by customer_id, year
    ),
    
    check as (
        select
            *,
            price > lag(price, 1, -1) over(partition by customer_id order by year asc) as is_bigger_last
        from (
            select
                cy.customer_id,
                cy.year,
                ifnull(cp.price, 0) as price
            from cus_year cy left join cus_year_price cp on cy.customer_id = cp.customer_id and cy.year = cp.year 
            order by cy.customer_id, cy.year
        )
    )
    
    select
        distinct 
        customer_id
    from check c where customer_id not in (select customer_id from check where is_bigger_last = False)
"""

In [23]:
spark.sql(query).show()

+-----------+
|customer_id|
+-----------+
|          1|
+-----------+

